In [1]:
import sys
import os
import subprocess
import glob

In [2]:
file_list = glob.glob('/scratch/ab9738/pollution_img/cctv_delhi/Raw/CCTV Images/*/*/*')

In [3]:
import pandas as pd
import numpy as np

In [4]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 30.8 MB/s eta 0:00:00


In [5]:
df = pd.read_excel('/scratch/ab9738/pollution_img/code/cctv_aqi.xlsx')

In [6]:
df[df['File_name'].duplicated()==True]

,File_name,ODE_ID,Camera_name,Camera_location,Camera_lat,Camera_lon,Station_name,Station_lat,Station_lon,Date,Time,PM 2.5,PM 10
886,chan01_20220420_050141.jpg,408511.0,chan01,"H. No. 822, Type-3, Mandir Marg, New Delhi",28.637574,77.202628,Mandir Marg,28.636424,77.201121,04-20-2022,05:01:41,107.0,268.0
905,chan02_20220420_120145.jpg,408511.0,chan02,"H. No. 822, Type-3, Mandir Marg, New Delhi",28.637574,77.202628,Mandir Marg,28.636424,77.201121,04-20-2022,12:01:45,73.0,223.0


In [7]:
for file in file_list:
    filename = file.split('/')[-3]+'_'+file.split('/')[-1]
    dest = '/scratch/ab9738/pollution_img/data/'+filename
    subprocess.call(['cp', file, dest])

In [8]:
for file in file_list:
    filename = file.split('/')[-1]
    extract = df[df['File_name']==filename]
    if(len(extract)==1):
        idx = extract.index[0]
        df.at[idx,'File_name'] = file.split('/')[-3]+'_'+filename

In [9]:
df_vals = df[['File_name','PM 2.5']].dropna()

In [10]:
df_vals = df_vals.drop(df_vals[df_vals['File_name'].str[0]=='c'].index)

In [11]:
df_vals.columns = ['filename','ppm']

In [12]:
df_vals = df_vals.reset_index(drop=True)

In [13]:
df_vals['filename'] = df_vals['filename'].str[:-4]

In [14]:
df_vals

,filename,ppm
0,"DITE, Okhla Industrial Area_523324_chan01_2022...",56.0
1,"DITE, Okhla Industrial Area_523324_chan01_2022...",39.0
2,"DITE, Okhla Industrial Area_523324_chan01_2022...",38.0
3,"DITE, Okhla Industrial Area_523324_chan01_2022...",44.0
4,"DITE, Okhla Industrial Area_523324_chan01_2022...",39.0
...,...,...
890,N.P. Boys Sr. Sec. School_408511_chan03_202204...,159.0
891,N.P. Boys Sr. Sec. School_408511_chan03_202204...,174.0
892,N.P. Boys Sr. Sec. School_408511_chan03_202204...,141.0
893,N.P. Boys Sr. Sec. School_408511_chan03_202204...,111.0


In [66]:
all_data = pd.read_csv('/scratch/ab9738/pollution_img/code/all_data.csv')

In [67]:
all_data

,filename,ppm
0,50294736_435828830291236_2030158088675541642_n,138.065714
1,23279786_148197742578856_6378254909640278016_n,523.672857
2,23416386_1840710242870758_1418351604486111232_n,523.672857
3,23279242_754613681413241_2636689794633039872_n,523.672857
4,23421744_157913204814458_5150676316399337472_n,205.317500
...,...,...
4344,Beijing/2014_11_27_0732_mod,14.000000
4345,Beijing/2014_12_27_0806_mod,182.000000
4346,Beijing/2014_03_05_0710_mod,35.000000
4347,Beijing/2014_03_18_0701_mod,20.000000


In [15]:
train_cctv = df_vals.sample(frac=0.8,random_state=17)
test_cctv = df_vals.loc[~df_vals.index.isin(train_cctv.index)]

In [16]:
train_cctv

,filename,ppm
644,RK Puram_440526_chan02_20220315_010141,168.0
561,RK Puram_440526_chan01_20220309_040139,100.0
261,Sonia Vihar Water Treatment Plant_701135_chan0...,45.0
69,"Sector 8, Dwarka_361115_chan01_20211011_221908",88.0
472,Sri Aurobindo Marg_456540_chan02_20220315_015035,106.0
...,...,...
373,Sonia Vihar Water Treatment Plant_701135_chan0...,119.0
497,Sri Aurobindo Marg_456540_chan02_20220316_005027,95.0
405,Sonia Vihar Water Treatment Plant_701135_chan0...,55.0
302,Sonia Vihar Water Treatment Plant_701135_chan0...,66.0


In [18]:
# train_df = pd.concat([train_cctv, all_data], axis=0)
train_df = train_cctv.reset_index(drop=True)
train_df.to_csv('/scratch/ab9738/pollution_img/code/train_data.csv', index=False)

In [19]:
test_df = test_cctv.reset_index(drop=True)
test_df.to_csv('/scratch/ab9738/pollution_img/code/test_data.csv', index=False)